In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import cv2
import os
from collections import defaultdict
import time

from skimage.feature import hog
from skimage import exposure

from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (
    classification_report, 
    confusion_matrix,
    precision_recall_fscore_support,
    accuracy_score,
    roc_auc_score
)

from tqdm.notebook import tqdm

#### Data  path configuration 

In [ ]:

base_dir = Path('../data')
train_dir = base_dir / 'train'
test_dir = base_dir / 'test'
validation_dir = base_dir / 'valid'

classes = [ "Ants",
    "Bees",
    "Beetles",
    "Caterpillars",
    "Earthworms",
    "Earwigs",
    "Grasshoppers",
    "Moths",
    "Slugs",
    "Snails",
    "Wasps",
    "Weevils"]
 
num_classes = len(classes)

#### Data Preprocessing

In [4]:
def parse_yolo_label(label_path):
    boxes = []
    
    if not os.path.exists(label_path):
        return boxes
    
    with open(label_path, 'r') as f:
        for line in f:
            line = line.strip()
            if line:
                parts = line.split()
                class_id = int(parts[0])
                x_center = float(parts[1])
                y_center = float(parts[2])
                width = float(parts[3])
                height = float(parts[4])
                boxes.append([class_id, x_center, y_center, width, height])
    
    return boxes

def yolo_to_bbox(x_center, y_center, width, height, img_width, img_height):
    x_center_abs = x_center * img_width
    y_center_abs = y_center * img_height
    width_abs = width * img_width
    height_abs = height * img_height
    
    x_min = int(x_center_abs - width_abs / 2)
    y_min = int(y_center_abs - height_abs / 2)
    x_max = int(x_center_abs + width_abs / 2)
    y_max = int(y_center_abs + height_abs / 2)
    
    return (x_min, y_min, x_max, y_max)


def crop_object_from_image(image, bbox):
    x_min, y_min, x_max, y_max = bbox
    
    h, w = image.shape[:2]
    x_min = max(0, x_min)
    y_min = max(0, y_min)
    x_max = min(w, x_max)
    y_max = min(h, y_max)
    
    cropped = image[y_min:y_max, x_min:x_max]
    
    if cropped.shape[0] < 10 or cropped.shape[1] < 10:
        return None
    
    return cropped

def load_yolo_dataset(data_dir, max_samples=None, use_full_image=False):
    images_dir = data_dir / 'images'
    labels_dir = data_dir / 'labels'
    
    print(f"Loading data from {data_dir}...")
    
    images = []
    labels = []
    bboxes = []
    
    image_files = sorted(list(images_dir.glob('*.jpg')))
    
    if max_samples:
        image_files = image_files[:max_samples]
    
    for img_path in image_files:
        img = cv2.imread(str(img_path))
        if img is None:
            continue
        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_height, img_width = img.shape[:2]
           
        label_path = labels_dir / (img_path.stem + '.txt')
        
        if not label_path.exists():
            continue
        
        boxes = parse_yolo_label(label_path)
        
        if len(boxes) == 0:
            continue
        
        for box in boxes:
            class_id, x_c, y_c, w, h = box
            
            if class_id >= num_classes:
                continue  
            
            bbox = yolo_to_bbox(x_c, y_c, w, h, img_width, img_height)
            
            if use_full_image:
                cropped = img
            else:
                cropped = crop_object_from_image(img, bbox)
                
                if cropped is None:
                    continue
            
            images.append(cropped)
            labels.append(classes[class_id])
            bboxes.append(bbox)
    
    print(f"Loaded {len(images)} objects from {len(image_files)} images")
    
    return images, labels, bboxes


#### HOG Feature Extraction

In [ ]:


class HOGFeatureExtractor:
    def __init__(self, image_size=(128, 128), orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2)):
        self.image_size = image_size
        self.hog_params = {
            'orientations': orientations,
            'pixels_per_cell': pixels_per_cell,
            'cells_per_block': cells_per_block,
            'block_norm': 'L2-Hys',
            'channel_axis': None
        }

    def preprocess_image(self, image):
        if len(image.shape) ==3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()

        resized = cv2.resize(gray, self.image_size)
        return resized
    
    def extract_features(self, images):
        single_image = False
        if not isinstance(images, list):
            images = [images]
            single_image = True 
        
        features_list = []
        iterator = tqdm(images, desc="Extracting HOG")

        for img in images:
            preprocessed = self.preprocess_image(img)
            feature = hog(preprocessed, visualize=False, **self.hog_params)
            features_list.append(feature)
        
        features = np.array(features_list)

        return features[0] if single_image else features

    def visualize(self, image):
        preprocessed = self.preprocess_image(image)
        
        hog_params_viz = self.hog_params.copy()
        
        features, hog_image = hog(
            preprocessed, 
            visualize=True, 
            **hog_params_viz
        )
        
        hog_image = exposure.rescale_intensity(hog_image, in_range=(0, 10))
        
        return features, hog_image, preprocessed

#### SVM Classifier


In [17]:

class SVMClassifier:
    def __init__(self, kernel='rbf', C=1.0, gamma='scale'):
        self.kernel = kernel
        self.C = C
        self.gamma = gamma
        
        self.model = None
        self.scaler = StandardScaler()
        self.label_encoder = LabelEncoder()
        
        self.training_time = None
        self.testing_time = None

    def train(self, X_train, y_train):
        print(f"Training SVM (kernel={self.kernel}, C={self.C})...")
        print(f"Training samples: {len(X_train)}")

        start_time = time.time()

        # Feature scaling (fit on training data)
        X_train_scaled = self.scaler.fit_transform(X_train)
        
        # Label encoding
        y_train_encoded = self.label_encoder.fit_transform(y_train)
        
        # Check class distribution
        unique, counts = np.unique(y_train_encoded, return_counts=True)
        print(f"   Classes: {len(unique)}, Distribution: {dict(zip(unique, counts))}")
        
        # SVM training
        self.model = SVC(
            kernel=self.kernel,
            C=self.C,
            gamma=self.gamma,
            probability=True,  
            random_state=42,
            verbose=False
        )

        self.model.fit(X_train_scaled, y_train_encoded)
        
        self.training_time = time.time() - start_time
        self.is_trained = True
        
        print(f"Training completed in {self.training_time:.2f} seconds")
        print(f"Support vectors: {self.model.n_support_}")

    def predict(self, X_test):
        X_test_scaled = self.scaler.transform(X_test)
        y_pred_encoded = self.model.predict(X_test_scaled)
        return self.label_encoder.inverse_transform(y_pred_encoded)
    
    def predict_prob(self, X_test):
        X_test_scaled = self.scaler.transform(X_test)
        return self.model.predict_prob(X_test_scaled)

    def evaluate(self, X_test, y_test, dataset_name='Test'):
        print(f"Evaluating on {dataset_name} set...")
        
        start_time = time.time()

        y_pred = self.predict(X_test)
        y_test_encoded = self.label_encoder.transform(y_test)
        y_pred_encoded = self.label_encoder.transform(y_pred)

        pass #do it later.. (nov 4)


